# Import Event Log

In [39]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/RequestForPayment.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

parsing log, completed traces ::   0%|          | 0/6886 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:Rfp_id,case:Project,case:Task,case:concept:name,case:OrganizationalEntity,case:Cost Type,case:RequestedAmount,case:Activity,case:RfpNumber,@@case_index
0,st_step 148220_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-09 08:17:18+00:00,EMPLOYEE,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
1,st_step 148221_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-09 08:18:00+00:00,SUPERVISOR,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
2,st_step 148222_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-10 11:42:32+00:00,MISSING,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
3,st_step 148219_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-03-03 08:51:13+00:00,EMPLOYEE,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
4,st_step 148218_0,STAFF MEMBER,Request For Payment APPROVED by PRE_APPROVER,2017-03-03 08:51:42+00:00,PRE_APPROVER,request for payment 148214,project 148216,UNKNOWN,request for payment 148214,organizational unit 65463,0,34.336343,UNKNOWN,request for payment number 148215,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36791,st_step 185004_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-29 11:35:02+00:00,ADMINISTRATION,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885
36792,st_step 185003_0,STAFF MEMBER,Request For Payment APPROVED by BUDGET OWNER,2019-01-03 08:27:20+00:00,BUDGET OWNER,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885
36793,st_step 185005_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-08 08:00:39+00:00,SUPERVISOR,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885
36794,rp_request for payment 185000_15,SYSTEM,Request Payment,2019-01-08 08:29:14+00:00,UNDEFINED,request for payment 185000,project 147860,task 152704,request for payment 185000,organizational unit 65468,0,15.409660,activity 505,request for payment number 185001,6885


# Preprocess

In [40]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

# Generate Prefixes

In [41]:
df_activity = dataframe_log[['concept:name', '@@case_index']]

In [42]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array

In [43]:
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [44]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, BatchNormalization

def create_binetv3_for_activity(num_activities, embedding_dim, gru_units, dropout_rate):
    # Input layers for activity attribute
    input_activity = Input(shape=(None,), name='activity_input')

    # Embedding layer for activity attribute
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)

    # Encoder GRU with Batch Normalization for activity
    encoded_activity = GRU(units=gru_units, return_sequences=False, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)

    # Dropout layer
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(bn_activity)

    # Output layer for predicting the next activity
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)

    # Building the model
    model = Model(inputs=input_activity, outputs=output_activity)
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_activities = dataframe_log['concept:name'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3_for_activity(num_activities, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activity_input (InputLayer  [(None, None)]            0         
 )                                                               
                                                                 
 activity_embedding (Embedd  (None, None, 50)          1000      
 ing)                                                            
                                                                 
 activity_encoder (GRU)      (None, 40)                11040     
                                                                 
 bn_activity (BatchNormaliz  (None, 40)                160       
 ation)                                                          
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                           

### Data Splitting

In [46]:
from sklearn.model_selection import train_test_split

train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

### Training

In [47]:
from tensorflow.keras.utils import to_categorical

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

In [48]:
history = model.fit(
    train_activity,  # Only the activity input
    train_targets_activity_cat,  # Only the activity targets
    validation_data=(test_activity, test_targets_activity_cat),  # Only the activity input and targets for validation
    epochs=20,
    batch_size=500
)

Epoch 1/20
42/42 [==============================] - 7s 79ms/step - loss: 1.5749 - accuracy: 0.6814 - val_loss: 2.6668 - val_accuracy: 0.6595
Epoch 2/20
42/42 [==============================] - 1s 32ms/step - loss: 0.5442 - accuracy: 0.8506 - val_loss: 2.4321 - val_accuracy: 0.6898
Epoch 3/20
42/42 [==============================] - 1s 30ms/step - loss: 0.4617 - accuracy: 0.8527 - val_loss: 2.2299 - val_accuracy: 0.8442
Epoch 4/20
42/42 [==============================] - 1s 32ms/step - loss: 0.4420 - accuracy: 0.8534 - val_loss: 2.0119 - val_accuracy: 0.8461
Epoch 5/20
42/42 [==============================] - 1s 31ms/step - loss: 0.4348 - accuracy: 0.8536 - val_loss: 1.7767 - val_accuracy: 0.8461
Epoch 6/20
42/42 [==============================] - 1s 31ms/step - loss: 0.4260 - accuracy: 0.8546 - val_loss: 1.5162 - val_accuracy: 0.8461
Epoch 7/20
42/42 [==============================] - 1s 31ms/step - loss: 0.4219 - accuracy: 0.8550 - val_loss: 1.2488 - val_accuracy: 0.8461
Epoch 8/20
42

In [49]:
# Evaluate the model on the validation set
results = model.evaluate(
    test_activity,  # Only the activity input
    test_targets_activity_cat,  # Only the activity targets
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

141/141 [==============================] - 0s 2ms/step - loss: 0.4178 - accuracy: 0.8479
Validation Loss: 0.41779589653015137, Validation Accuracy: 0.8478769659996033


In [50]:
# Save the model to an H5 file
model.save('RfP.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

- NOTE: Do not include ID because no learning through GRU

In [51]:
# Generate predictions for all inputs
predictions_activity = model.predict([windows_activity])

935/935 [==============================] - 3s 2ms/step


In [52]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [53]:
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))

## Insert missing scores for cases with less than 2 Events

In [54]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_activity'] = anomaly_scores_activity

score['case'] = score['case'].astype(int)

score

,case,score_activity
0,0,0.855848
1,0,0.954231
2,0,0.000000
3,0,0.725297
4,0,0.000000
...,...,...
29905,6885,0.000000
29906,6885,0.636433
29907,6885,0.000000
29908,6885,0.000000


In [55]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(0, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 4999

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")

Missing values: [179, 181, 188, 354, 387, 450, 529, 545, 601, 765, 835, 974, 1020, 1202, 1273, 1299, 1301, 1324, 1329, 1347, 1390, 1431, 1578, 1601, 1636, 1895, 2022, 2091, 2151, 2152, 2226, 2367, 2466, 2712, 2897, 2983, 3041, 3297, 3440, 3478, 3573, 3652, 3659, 3865, 3960, 4064, 4235, 4279, 4389, 4638, 4865]
Does the 'case' column contain all values between 0 and 4999? False


In [56]:
# Generate the full range of values
full_range = set(range(6886))

# Get the existing 'case' values
existing_cases = set(score['case'])

# Identify missing values
missing_cases = full_range - existing_cases

# Create a DataFrame for missing values
missing_df = pd.DataFrame({'case': list(missing_cases)})

# Add other columns with value 1
for col in score.columns:
    if col != 'case':
        missing_df[col] = 1

# Concatenate the original DataFrame with the missing values DataFrame
updated_score = pd.concat([score, missing_df])

# Sort the DataFrame by 'case'
updated_score = updated_score.sort_values(by='case').reset_index(drop=True)

updated_score

,case,score_activity
0,0,0.855848
1,0,0.954231
2,0,0.000000
3,0,0.725297
4,0,0.000000
...,...,...
29977,6885,0.000000
29978,6885,0.636433
29979,6885,0.000000
29980,6885,0.000000


In [57]:
anomaly_scores_activity = updated_score['score_activity']

### Threshold (lowest plateau)

In [58]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [59]:
threshold_activity = find_plateaus(anomaly_scores_activity)

In [60]:
anomaly_scores_activity

0        0.855848
1        0.954231
2        0.000000
3        0.725297
4        0.000000
           ...   
29977    0.000000
29978    0.636433
29979    0.000000
29980    0.000000
29981    0.000000
Name: score_activity, Length: 29982, dtype: float64

In [61]:
threshold_activity

0.6364331245422363

### Detection

In [62]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [63]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)

# Mapping

In [64]:
case_indices = updated_score['case']

In [65]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_activity'] = labels_activity

mapping

,case,predicted_activity
0,0,1
1,0,1
2,0,0
3,0,1
4,0,0
...,...,...
29977,6885,0
29978,6885,0
29979,6885,0
29980,6885,0


In [66]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_activity']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0        True
1        True
2        True
3        True
4        True
        ...  
6881    False
6882    False
6883    False
6884    False
6885    False
Length: 6886, dtype: bool

# Ground Truth

In [67]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [68]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_RequestForPayment.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/89 [00:00<?, ?it/s]

In [69]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [70]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
6881,0,0
6882,0,0
6883,0,0
6884,0,0


# Evaluation

In [71]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [72]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.80


In [73]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 1.00


In [74]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 1.00


In [75]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.92


In [76]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.96
